In [1]:
# Libraries
import pandas as pd
import numpy as np

In [2]:
# Global data import for all viz's
raw = pd.read_csv("https://github.com/zmosDS/recession_impact_analysis/releases/download/1/bls_employment_stats.csv")

raw.head()

,series_id,industry_name,supersector_name,naics_code,year,month,value,seasonal,month_num
0,CES0000000001,Total nonfarm,Total nonfarm,-,1995,January,116505.0,S,1
1,CES0000000001,Total nonfarm,Total nonfarm,-,1995,February,116694.0,S,2
2,CES0000000001,Total nonfarm,Total nonfarm,-,1995,March,116908.0,S,3
3,CES0000000001,Total nonfarm,Total nonfarm,-,1995,April,117065.0,S,4
4,CES0000000001,Total nonfarm,Total nonfarm,-,1995,May,117047.0,S,5


In [ ]:
# Filtering for YoY change (Visualization 1)

# Filter for total nonfarm
nonfarm = raw[raw["industry_name"] == "Total nonfarm"].copy()

# Compute annual averages
annual = nonfarm.groupby("year")["value"].mean().reset_index()

# Compute YoY percent change
annual["yoy_change"] = annual["value"].pct_change() * 100

# Save
annual.to_csv("viz1_yoy_change.csv", index=False)